In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%cd /home/adityasidharta/git/shopee_data_science
%env PROJECT_PATH /home/adityasidharta/git/shopee_data_science

In [ ]:
from tqdm import tqdm
import argparse
from sklearn.externals import joblib
from fastai.vision import *

from model.image.fastai.ml_model import fastai_prediction
from model.common.topic import beauty_columns, mobile_columns, fashion_columns
from utils.envs import *
from utils.logger import logger
from utils.common import get_datetime, create_directory

In [ ]:
def fix_image_path(input_df, relative_path):
    df = input_df.copy()
    for idx in tqdm(range(len(input_df))):
        filename = df.at[idx, 'image_path'].split('/')[1]
        if not filename.endswith('.jpg'):
            filename = filename + '.jpg'
        final_filename = os.path.join(relative_path, filename)
        df.at[idx, 'image_path'] = final_filename
    return df


In [ ]:
logger.setup_logger('img_fastai')
datetime = get_datetime()
model_path = os.path.join(result_path, 'img_fastai_{}').format(datetime)
model_metadata_path = os.path.join(result_metadata_path, 'img_fastai_{}').format(datetime)

In [ ]:
create_directory(model_path)
create_directory(model_metadata_path)

In [ ]:
path = Path(img_root)

beauty_dev = pd.read_csv(beauty_dev_repo)
beauty_val = pd.read_csv(beauty_val_repo)
beauty_test = pd.read_csv(beauty_test_repo)

fashion_dev = pd.read_csv(fashion_dev_repo)
fashion_val = pd.read_csv(fashion_val_repo)
fashion_test = pd.read_csv(fashion_test_repo)

mobile_dev = pd.read_csv(mobile_dev_repo)
mobile_val = pd.read_csv(mobile_val_repo)
mobile_test = pd.read_csv(mobile_test_repo)

beauty_dev = fix_image_path(beauty_dev, 'beauty_dev')
beauty_val = fix_image_path(beauty_val, 'beauty_val')
beauty_test = fix_image_path(beauty_test, 'beauty_test')

fashion_dev = fix_image_path(fashion_dev, 'fashion_dev')
fashion_val = fix_image_path(fashion_val, 'fashion_val')
fashion_test = fix_image_path(fashion_test, 'fashion_test')

mobile_dev = fix_image_path(mobile_dev, 'mobile_dev')
mobile_val = fix_image_path(mobile_val, 'mobile_val')
mobile_test = fix_image_path(mobile_test, 'mobile_test')

In [ ]:
train_beauty = pd.concat([beauty_dev, beauty_val]).iloc[0:300, :]
test_beauty = beauty_test
test_beauty_folder = 'beauty_try'

train_fashion = pd.concat([fashion_dev, fashion_val]).iloc[0:64, :]
test_fashion = fashion_test
test_fashion_folder = 'fashion_test'

train_mobile = pd.concat([mobile_dev, mobile_val]).iloc[0:64, :]
test_mobile = mobile_test
test_mobile_folder = 'mobile_test'

In [ ]:
train_beauty

In [ ]:
beauty_result_dict = fastai_prediction(train_beauty, test_beauty, test_beauty_folder, ['Brand', 'Benefits'], path, 'beauty')